In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp plugin.pluginbase

In [ ]:
# export
from pymemri.data.schema import *
from pymemri.pod.client import *
from pymemri.imports import *
from pymemri.plugin.states import *
from pymemri.pod.utils import *
from pymemri.plugin.listeners import get_abort_plugin_listener

from os import environ
from abc import ABCMeta
import abc
import json
import importlib
import string
import time
from enum import Enum
from fastscript import *
import os
from pymemri.plugin.schema import Account, PluginRun
from pymemri.data.basic import *
from pymemri.pod.client import Dog, PodClient, DEFAULT_POD_ADDRESS
import warnings
from pymemri.data.basic import write_json
from pymemri.plugin.authenticators.credentials import PLUGIN_DIR
from fastcore.script import call_parse, Param
import os
import traceback

In [ ]:
# hide
from nbdev.showdoc import *

# Plugins

PluginBase is the plugin class that the simplest plugin inherits.

Inheriting class should implement:

- run()           that implements the logic of the plugin
- add_to_schema() for plugin specific item types

In [ ]:
# export
POD_FULL_ADDRESS_ENV        = 'POD_FULL_ADDRESS'
POD_TARGET_ITEM_ENV         = 'POD_TARGET_ITEM'
POD_OWNER_KEY_ENV           = 'POD_OWNER'
POD_AUTH_JSON_ENV           = 'POD_AUTH_JSON'
POD_PLUGIN_DNS_ENV          = 'PLUGIN_DNS'

In [ ]:
# export
# hide
class PluginBase(metaclass=ABCMeta):
    """Base class for plugins"""

    def __init__(self, pluginRun=None, client=None, **kwargs):
        super().__init__()
        if pluginRun is None:
            warnings.warn(
                "Plugin needs a pluginRun as kwarg, running without will only work in development.",
                RuntimeWarning)
        self.pluginRun = pluginRun
        
        if client is None:
            raise ValueError("Plugins need a `client: PodClient` as kwarg to run.")
        self.client = client
        self._status_listeners = []
        self._config_dict = kwargs

    def set_run_status(self, status):
        # TODO sync before setting status (requires pod_client.sync())
        if self.pluginRun and self.client:
            self.pluginRun.status = status
            self.client.update_item(self.pluginRun)

    def setup(self):
        if self.client and self.pluginRun:
            status_abort_listener = get_abort_plugin_listener(self.client, self.pluginRun.id)
            self._status_listeners.append(status_abort_listener)
    
    def teardown(self):
        for listener in self._status_listeners:
            listener.stop()
        
    def _run(self):
        self.setup()
        self.run()
        self.teardown()

    @abc.abstractmethod
    def run(self):
        raise NotImplementedError()

    @abc.abstractmethod
    def add_to_schema(self):
        """
        Add all schema classes required by the plugin to self.client here.
        """
        raise NotImplementedError()

In [ ]:
# export
# hide
class PluginError(Exception):
    """Generic class for plugin errors. This error is raised when a plugin raises an unexpected exception."""
    pass

## Creating a plugin

The memri [pod](https://gitlab.memri.io/memri/pod) uses a plugin system to add features to the backend memri backend. Plugins can import your data (importers), change your data (indexers), or call other serivces. Users can define their own plugins to add new behaviour to their memri app. Let's use the following plugin as an example of how we can start plugins.

```python
class ExamplePlugin(PluginBase):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def run(self):
        print("running")
        self.client.create(Dog("some dog", 20))

    def add_to_schema(self):
        self.client.add_to_schema(Dog("my name", 10))
```

In [ ]:
# export
# hide
class ExamplePlugin(PluginBase):
    """"""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def run(self):
        self.client.create(Dog("some dog", 20))
        print("Plugin run success.")

    def add_to_schema(self):
        self.client.add_to_schema(Dog("my name", 10))

Memri plugins need to define at least 2 methods: `.run()` and `.add_to_schema()`. `.run()` defines the logic of the plugin. `.add_to_schema()` defines the schema for the plugin in the pod. Note that currently, `add_to_schema` requires all item to **have all properties defined that are used in the plugin**. In the future, we might replace add_to_schema, to be done automatically, based on a declarative schema defined in the plugin. 

### Authentication

Many plugins use authentication. For examples, see `OAuthAuthenticator` or `PasswordAuthenticator`.

## Helper methods

In [ ]:
# export
# hide
def write_run_info(plugin, id_):
    try:
        if plugin is None:
            raise ValueError("Empty container")
        run_path = PLUGIN_DIR / plugin / "current_run.json"
        run_path.parent.mkdir(parents=True, exist_ok=True)
        print(f"writing run info to {run_path}")
        write_json({"id": id_}, run_path)
    except Exception as e:
        print(f"""failed to write run info to {run_path}\n{e}""")

In [ ]:
# hide
# export
def get_plugin_cls(plugin_module, plugin_name):
    try:
        module = importlib.import_module(plugin_module)
        plugin_cls = getattr(module, plugin_name)
        return plugin_cls
    except (ImportError, AttributeError):
        raise ImportError(f"Unknown plugin: {plugin_module}.{plugin_name}")

def run_plugin_from_run_id(run_id, client, **kwargs):
    """
    Runs a plugin from run_id, initialized with **kwargs.
    
    Args:
        client (PodClient): client containing PluginRun
        run_id (int): id of the PluginRun
    """

    run = client.get(run_id)
    write_run_info(run.pluginModule.split(".")[0] if run.pluginModule is not None else run.containerImage, run.id)

    plugin_cls = get_plugin_cls(run.pluginModule, run.pluginName)
    plugin = plugin_cls(pluginRun=run, client=client, **kwargs)
    plugin.add_to_schema()

    plugin.set_run_status(RUN_STARTED)
    plugin._run()
    plugin.pluginRun = plugin.client.get(run_id)
    plugin.set_run_status(RUN_COMPLETED)

    return plugin

## Run from id test -

In [ ]:
# hide
# skip
client = PodClient()
run = PluginRun(
    containerImage="pymemri",
    pluginModule="pymemri.plugin.pluginbase",
    pluginName="ExamplePlugin",
    status="not started")
account = Account(identifier="login", secret="password")
run.add_edge("account", account)
assert client.add_to_schema(PluginRun("", "", "", "", ""))
assert client.create(run)
assert client.create(account)
assert client.create_edge(run.get_edges("account")[0])

print(run.to_json())

run_plugin_from_run_id(run.id, client);

run = client.get(run.id)
assert run.status == RUN_COMPLETED

{'id': '3BFcDbf6b42FC923fEbe4FeDBfF04feF', 'containerImage': 'pymemri', 'pluginModule': 'pymemri.plugin.pluginbase', 'pluginName': 'ExamplePlugin', 'status': 'not started', 'targetItemId': '3BFcDbf6b42FC923fEbe4FeDBfF04feF', 'type': 'PluginRun'}
writing run info to /home/eelco/.pymemri/plugins/pymemri/current_run.json


SyntaxError: invalid syntax (pluginbase.py, line 309)

In [ ]:
# export
# hide
def _parse_env():
    env = os.environ
    print("Reading `run_plugin()` parameters from environment variables")
    try:
        pod_full_address = env.get(POD_FULL_ADDRESS_ENV, DEFAULT_POD_ADDRESS)
        plugin_run_json  = json.loads(str(env[POD_TARGET_ITEM_ENV]))
        print(plugin_run_json)
        plugin_run_id    = plugin_run_json["id"]
        owner_key        = env.get(POD_OWNER_KEY_ENV)
        pod_auth_json    = json.loads(str(env.get(POD_AUTH_JSON_ENV)))
        return pod_full_address, plugin_run_id, pod_auth_json, owner_key
    except KeyError as e:
        raise Exception('Missing parameter: {}'.format(e)) from None

## Running your plugin using the CLI

Plugins can be started using the pymemri `run_plugin` or `simulate_run_plugin_from_frontend` CLI. With `run_plugin` the plugin is invoked directly by spawning a new python process, while `simulate_run_plugin_from_frontend` requests the pod to spawn a new process, docker container, or kubernetes container, which in calls `run_plugin` (for more info see `simulate_run_plugin_from_frontend`. When using `run_plugin`, you can either pass your run arguments as parameters, or set them as environment variables. If both are set, the CLI will use the passed arguments.

In [ ]:
# export
# hide
@call_parse
def store_keys(path:Param("path to store the keys", str)=DEFAULT_POD_KEY_PATH,
               database_key:Param("Database key of the pod", str)=None,
               owner_key:Param("Owner key of the pod", str)=None):
    
    if database_key is None: database_key = PodClient.generate_random_key()
    if owner_key is None: owner_key = PodClient.generate_random_key()

    obj = {"database_key": database_key,
           "owner_key": owner_key}
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    if path.exists():
        timestr = time.strftime("%Y%m%d-%H%M%S")
        path.rename(POD_KEYS_FULL_FOLDER / f"keys-{timestr}.json")
    write_json(obj, path)

In [ ]:
# hide
store_keys()

In [ ]:
# export
# hide
def parse_metadata(fn, remove_container=False):
    metadata = read_json(fn)
    for k in ["pluginModule", "pluginName"]:
        if k not in metadata:
            raise ValueError(f"Missing metadata: {k}")
            
    run_vars = {k: v for k, v in metadata.items() if k in PluginRun.properties}
    run = PluginRun.from_json(run_vars)
    if remove_container:
        run.containerImage = "none"
        
    if "account" in metadata:
        account = Account.from_json(metadata["account"])
        run.add_edge("account", account)
    return run


def parse_config(run_config, config_file=None, remove_container=False):
    """
    Parse the configuration of the plugin. A configuration is a dict that is passed to the plugin init as kwargs.
    If configuration file is defined, the run_config is ignored.
    """
    if config_file is not None:
        config = read_json(config_file)
    elif isinstance(run_config, str) and len(run_config):
        config = json.loads(run_config)
    else:
        config = dict()
    
    if not isinstance(config, dict):
        raise ValueError(f"Incorrect plugin config format, expected a dict, got a {type(config)}")
    return config


def create_run_expanded(client, run):
    client.create(run)
    accounts = run.account
    if accounts:
        account=accounts[0]
        client.create(account)
        client.create_edge(run.get_edges("account")[0])    

In [ ]:
# export
@call_parse
def run_plugin(
    pod_full_address: Param("The pod full address", str) = DEFAULT_POD_ADDRESS,
    plugin_run_id: Param("Run id of the plugin to be executed", str) = None,
    database_key: Param("Database key of the pod", str) = None,
    owner_key: Param("Owner key of the pod", str) = None,
    read_args_from_env: Param("Read the args from the environment", bool) = False,
    metadata: Param("metadata file for the PluginRun", str) = None,
    config_file: Param(
        "A plugin configuration, overwrites the configuration of the PluginRun", str
    ) = None,
):

    if read_args_from_env:
        pod_full_address, plugin_run_id, pod_auth_json, owner_key = _parse_env()
        database_key = None
    else:
        if database_key is None:
            database_key = read_pod_key("database_key")
        if owner_key is None:
            owner_key = read_pod_key("owner_key")
        pod_auth_json = None
    if POD_PLUGIN_DNS_ENV in os.environ:
        print(f"Plugin accesible via {os.environ.get(POD_PLUGIN_DNS_ENV)}:8080")

    client = PodClient(
        url=pod_full_address,
        database_key=database_key,
        owner_key=owner_key,
        auth_json=pod_auth_json,
    )
    print(f"pod_full_address={pod_full_address}\nowner_key={owner_key}\n")

    if metadata is not None:
        run = parse_metadata(metadata, remove_container=True)
        create_run_expanded(client, run)
        plugin_run_id = run.id
    else:
        run = client.get(plugin_run_id)
    plugin_config = parse_config(run.config, config_file)

    try:
        run_plugin_from_run_id(
            plugin_run_id, client, **plugin_config
        )
    except Exception as e:
        run = client.get(plugin_run_id)
        run.status = RUN_FAILED
        client.update_item(run)
        print(traceback.format_exc(), flush=True)
        raise PluginError("The plugin quit unexpectedly.") from None

To start a plugin on your local machine, you can use the CLI. This will create a client for you, and run the code defined in `<myplugin>.run()`

In [ ]:
run_plugin(metadata="../example_plugin.json")

> Note: The data that is created here should be in the pod in order for this to work

## Run from pod 

In production, we start plugins by making an API call to the pod, which in turn creates an environment for the plugin and starts it using docker containers, kubernetes containers or a shell script. We can start this process using the `simulate_run_plugin_from_frontend` CLI. **Note that when using docker, provided container name should be "installed" within the Pod environemnt (e.g. `docker build -t pymemri .` for this repo) in order to start it.** 

![running a plugin](images/running_a_plugin.svg)

In [ ]:
# export
@call_parse
def simulate_run_plugin_from_frontend(
    pod_full_address: Param("The pod full address", str) = DEFAULT_POD_ADDRESS,
    database_key: Param("Database key of the pod", str) = None,
    owner_key: Param("Owner key of the pod", str) = None,
    container: Param("Pod container to run frod", str) = None,
    plugin_path: Param("Plugin path", str) = None,
    metadata: Param("metadata file for the PluginRun", str) = None,
    config_file: Param(
        "A plugin configuration, overwrites the configuration of the PluginRun", str
    ) = None,
    account_id: Param("Account id to be used inside the plugin", str) = None,
):
    if database_key is None:
        database_key = read_pod_key("database_key")
    if owner_key is None:
        owner_key = read_pod_key("owner_key")
    params = [pod_full_address, database_key, owner_key]
    if None in params:
        raise ValueError(f"Missing Pod credentials")
    
    client = PodClient(url=pod_full_address, database_key=database_key, owner_key=owner_key)
    print(f"pod_full_address={pod_full_address}\nowner_key={owner_key}\n")
    
    if metadata is not None:
        run = parse_metadata(metadata)
        create_run_expanded(client, run)
    else:
        if container is None:
            container = plugin_path.split(".", 1)[0]
        print(f"Inferred '{container}' as plugin container name")
        plugin_module, plugin_name = plugin_path.rsplit(".", 1)
        run = PluginRun(container, plugin_module, plugin_name)

        if account_id is not None:
            account = client.get(account_id)
            run.add_edge("account", account)
            print(f"Using existing {account}")

        client.create(run)

    print(
        f"Created pluginrun with id {run.id} on {pod_full_address}"
    )

    plugin_dir = run.containerImage
    write_run_info(plugin_dir, run.id)

    print(f"*Check the pod log/console for debug output.*")
    return run

In [ ]:
client = PodClient()

In [ ]:
!simulate_run_plugin_from_frontend --metadata "../example_plugin.json"

reading database_key from /home/eelco/.pymemri/pod_keys/keys.json
reading owner_key from /home/eelco/.pymemri/pod_keys/keys.json
pod_full_address=http://localhost:3030
owner_key=0669504049511661364795770949641722972379709465482258592795264679

Created pluginrun with id DDDFb7dFDeFF5c5917e7f846BF5df099 on http://localhost:3030
writing run info to /home/eelco/.pymemri/plugins/pymemri/current_run.json
*Check the pod log/console for debug output.*


## Appendix -

In [ ]:
# hide
# client.start_plugin("pymemri", run.id)

In [ ]:
# hide
# # export
# def generate_test_env(client, indexer_run):
#     payload = json.dumps({DATABASE_KEY_ENV: client.database_key, OWNER_KEY_ENV: client.owner_key})
              
#     return {POD_FULL_ADDRESS_ENV: DEFAULT_POD_ADDRESS,
#             POD_TARGET_ITEM: indexer_run.id,
#             POD_SERVICE_PAYLOAD_ENV: payload}

In [ ]:
# hide
# run_plugin(env=generate_test_env(client, run))

# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted Untitled.ipynb.
Converted basic.ipynb.
Converted cvu.utils.ipynb.
Converted data.photo.ipynb.
Converted index.ipynb.
Converted itembase.ipynb.
Converted plugin.authenticators.credentials.ipynb.
Converted plugin.authenticators.oauth.ipynb.
Converted plugin.listeners.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.states.ipynb.
Converted plugins.authenticators.password.ipynb.
Converted pod.api.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
Converted template.config.ipynb.
Converted template.formatter.ipynb.
Converted test_schema.ipynb.
Converted test_utils.ipynb.
